In [1]:
from sqlalchemy import create_engine
import pandas as pd

# 1. Create the DB connection function
def create_db_connection(username, password, host, database):
    connection_str = f'mysql+pymysql://{username}:{password}@{host}/{database}'
    engine = create_engine(connection_str)
    return engine

# 2. Retrieve rental data for a specific month and year
def rentals_month(engine, month, year):
    query = f"""
        SELECT customer_id, rental_id, rental_date
        FROM rental
        WHERE MONTH(rental_date) = {month} AND YEAR(rental_date) = {year};
    """
    df = pd.read_sql(query, con=engine)
    return df

# 3. Count rentals for each customer in a given month and year
def rental_count_month(df, month, year):
    rental_count = df.groupby('customer_id').size().reset_index(name=f'rentals_{month:02}_{year}')
    return rental_count

# 4. Compare rental activity between two months
def compare_rentals(df_may, df_june):
    combined_df = pd.merge(df_may, df_june, on='customer_id', how='outer').fillna(0)
    may_col = df_may.columns[1]
    june_col = df_june.columns[1]
    combined_df['difference'] = combined_df[june_col] - combined_df[may_col]
    return combined_df

# Example of how the functions can be used (you will need the correct credentials)
# engine = create_db_connection("root", "password", "localhost", "sakila")
# may_rentals = rentals_month(engine, 5, 2005)
# june_rentals = rentals_month(engine, 6, 2005)
# may_rental_counts = rental_count_month(may_rentals, 5, 2005)
# june_rental_counts = rental_count_month(june_rentals, 6, 2005)
# comparison_df = compare_rentals(may_rental_counts, june_rental_counts)
